In [0]:
import pandas as pd
from keras.models import model_from_json
from sklearn.metrics import fbeta_score

import cv2

import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from keras.models import Sequential, load_model
from keras.layers import BatchNormalization, Activation, GlobalAveragePooling2D, UpSampling2D
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
from keras.applications.xception import Xception
from keras.applications.vgg16 import VGG16
from keras import backend as K

import matplotlib.pyplot as plt

import seaborn as sns

In [1]:
#@title
import tensorflow as tf
print(tf.test.gpu_device_name())

/device:GPU:0


In [2]:
#@title
! pip install pydrive

# these classes allow you to request the Google drive API
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive 
from google.colab import auth 
from oauth2client.client import GoogleCredentials
from google.colab import files

    100% |████████████████████████████████| 993kB 6.7MB/s 
  Running setup.py bdist_wheel for pydrive ... - \ done
  Stored in directory: /content/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built pydrive


In [0]:
#@title
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

to_download = {
    'fer2013.csv' : '1Yln-m9Fuo4fIJ0enTHUSw-Nf7Fq7gA1_',
    'arch_1.json' : '1CWcpm6x3dwR3Yv3ek6RH_66jblyEs2Fv',
    'arch_2.json' : '1D1gcWlMZwa1ZJXxQACnKKwDf_96z4fKE',
    'arch_3.json' : '1Medz2eqjH5q25TuCMSxWSldN3CLrk_tw',
    'arch_4.json' : '1tYzPlD5mQI7xJlm3GoNgwpFOQQqX1jat',
    'arch_5.json' : '1BgVCrgY8SdSdvMBBPhsRqVGv-H_dlTgs',
    'arch_6.json' : '1mUMG_DY_YT9fP_FLiFqOJc272fspg7gN',
    'arch_7.json' : '18aP5_NLlyoo1oVT0tj12oZjB5OIBupMy',
    'arch_8.json' : '1-w3xFnmqmO-OwXZ3lgLp66fpAieZQJC3',
    'arch_9.json' : '18W04KAFBSceYif0sGFhM4FPD3ZJjpKH6',
    'weights_1.h5' : '1GC5El9hPeW5SfDIahPd3-RcwtCFBV4rO',
    'weights_2.h5' : '1c8g9ijqg4WdtTUfygrWoahhEYbpVeYw6',
    'weights_3.h5' : '1LZ5jCc7wI9BhJMv2ejkSke5Uau8a9eoP',
    'weights_4.h5' : '1-7o_ROkJP06PT5XXCamf86_p2XviEBEC',
    'weights_5.h5' : '1RUNGVRGjL52FYN9FETbn-3UwXIq5HSPT',
    'weights_6.h5' : '1y4L9Fn70ddgGTn-INSAhF1qqzVTn7jDj',
    'weights_7.h5' : '14kFHdsO_D-JVYYUeDLJ0O-_pJ8Jcrn3K',
    'weights_8.h5' : '1YIi4FMuz5pfwmBl0E-kLm06fsXpCKEBR',
    'weights_9.h5' : '1LgC_eZQMoJ4Htjtau4wLz39lqpYFfcRa'
}

for filename, fileid in to_download.items():
  drive.CreateFile({'id': fileid}).GetContentFile(filename)

In [8]:
#@title
# load dataset from csv file
fer2013 = pd.read_csv("fer2013.csv")
fer2013.head()

emotions_labels_map = {'angry':0,
                       'disgust':1, 
                       'fear':2, 
                       'happy':3, 
                       'sad':4, 
                       'surprise':5, 
                       'neutral':6}

# fer2013.emotion = [1 if x == emotions_labels_map[EMOTION_TRAINED] else 0 for x in fer2013['emotion']]
print(fer2013.groupby(['emotion']).agg('count')['pixels'])

# group dataset by usage
groups = fer2013.groupby('Usage')
groups.count()

# get group - training

training_data = groups.get_group('Training')

# get groups on training split by emotion

training_emotions = training_data.groupby('emotion')
training_emotions.count()

# get group - test

test_data = groups.get_group('PublicTest')
# get groups on test split by emotion

test_emotions = test_data.groupby('emotion')
test_emotions.count()

# get group - validation

validation_data = groups.get_group('PrivateTest')

# get groups on validation split by emotion

validation_emotions = validation_data.groupby('emotion')
validation_emotions.count()

def prepare_dataset(dataframe, image_size={'width':48, 'height':48}):
    faces_images = []
    pixels = dataframe['pixels'].tolist()
    for pixel_sequence in pixels:
        face = [int(pixel) for pixel in pixel_sequence.split(' ')]
        face = np.asarray(face).reshape(image_size['width'], image_size['height'])
        face = cv2.resize(face.astype('uint8'), (image_size['width'], image_size['height']))
#         face = cv2.equalizeHist(face)
#         face = cv2.Laplacian(face, cv2.CV_64F)
        
        faces_images.append(face.astype('float32') / 255.0)
    faces_images = np.asarray(faces_images)
    faces_images = np.expand_dims(faces_images, -1) # (1, 48, 48)
    emotions_labels = pd.get_dummies(dataframe['emotion']).as_matrix()
    return faces_images, emotions_labels

# prepare dataset to training, test and validate model
training_faces, training_emotions = prepare_dataset(training_data)
print('training', len(training_faces), len(training_emotions))
test_faces, test_emotions = prepare_dataset(test_data)
print('test', len(test_faces), len(test_emotions))
validation_faces, validation_emotions = prepare_dataset(validation_data)
print('validation', len(validation_faces), len(validation_emotions))

# normalize faces

training_faces = training_faces.astype('float32')
test_faces = test_faces.astype('float32')
validation_faces = validation_faces.astype('float32')

# get size dataset elements
num_samples, num_classes = training_emotions.shape
print(num_samples, num_classes)

# split and randomize data
x_train_part1, x_train_part2, y_train_part1, y_train_part2 = train_test_split(training_faces, training_emotions, test_size=0.3, random_state=1)
x_train, y_train = np.vstack((x_train_part1, x_train_part2)), np.vstack((y_train_part1, y_train_part2))
print('training', len(x_train), len(y_train))

x_test_part1, x_test_part2, y_test_part1, y_test_part2 = train_test_split(test_faces, test_emotions, test_size=0.3, random_state=1)
x_test, y_test = np.vstack((x_test_part1, x_test_part2)), np.vstack((y_test_part1, y_test_part2))
print('test', len(x_test), len(y_test))

x_validation_part1, x_validation_part2, y_validation_part1, y_validation_part2 = train_test_split(validation_faces, validation_emotions, test_size=0.3, random_state=1)
x_validation, y_validation = np.vstack((x_validation_part1, x_validation_part2)), np.vstack((y_validation_part1, y_validation_part2))
print('validation', len(x_validation), len(y_validation))

def get_dataset():
    return x_train, y_train, x_test, y_test, x_validation, y_validation

emotion
0    4953
1     547
2    5121
3    8989
4    6077
5    4002
6    6198
Name: pixels, dtype: int64
training 28709 28709
test 3589 3589
validation 3589 3589
28709 7
training 28709 28709
test 3589 3589
validation 3589 3589


In [0]:
x_train, y_train, x_test, y_test, x_validation, y_validation = get_dataset()

In [0]:
MODELS_CNT = 9

cnn_models = []

for model_indice in range(1, MODELS_CNT + 1):
    with open('arch_{}.json'.format(model_indice)) as fl:
        model = model_from_json(fl.read())
    model.load_weights('weights_{}.h5'.format(model_indice))
    cnn_models.append(model)

In [13]:
for model_indice, model in zip(range(1, MODELS_CNT + 1), cnn_models):
    y_pred = model.predict_classes(x_validation)
    y_true = np.asarray([np.argmax(i) for i in y_validation])

    score = fbeta_score(y_true, y_pred, average='micro', beta=1.0)
    print(f"model: {model_indice} fbeta = {score}")

model: 1 fbeta = 0.6898857620507105
model: 2 fbeta = 0.6767901922541097
model: 3 fbeta = 0.6606297018668152
model: 4 fbeta = 0.6798551128448036
model: 5 fbeta = 0.6667595430482028
model: 6 fbeta = 0.6781833379771524
model: 7 fbeta = 0.6951797157982725
model: 8 fbeta = 0.6285873502368348
model: 9 fbeta = 0.6244079130677069


In [22]:
x_train_output = np.array(cnn_models[0].predict(x_train))
x_test_output = np.array(cnn_models[0].predict(x_test))
x_validation_output = np.array(cnn_models[0].predict(x_validation))
print('1', end='..')

for model_indice, model in zip(range(2, MODELS_CNT + 1), cnn_models[1:]):
  x_train_output = np.hstack((x_train_output, model.predict(x_train)))
  x_test_output = np.hstack((x_test_output, model.predict(x_test)))
  x_validation_output = np.hstack((x_validation_output, model.predict(x_validation)))
  print(f'{model_indice}', end='..')

1..2..3..4..5..6..7..8..9..

In [26]:
x_train_output.shape, x_test_output.shape, x_validation_output.shape

((28709, 63), (3589, 63), (3589, 63))

In [29]:
data_train = np.hstack( ( x_train_output, y_train ) )
data_test = np.hstack( ( x_test_output, y_test ) )
data_validation = np.hstack( ( x_validation_output, y_validation ) )

data_train.shape, data_test.shape, data_validation.shape

((28709, 70), (3589, 70), (3589, 70))

In [0]:
output_cnt = data_train.shape[1]
input_cnt = x_train_output.shape[1]

cols = [f'in{i}' if i < input_cnt else f'out{i}' for i in range(output_cnt)]

df = pd.DataFrame(data_train, columns=cols)
df.head()

df.to_csv('train.csv')

In [42]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

drive_service = build('drive', 'v3')

file_metadata = { 'name': 'train.csv', 'mimeType': 'text/csv' }
media = MediaFileUpload('train.csv', mimetype='text/csv', resumable=True)
created = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 10ObsG1CBpM6TM79h-Z6qJ04Xv44-iwBE


In [43]:
output_cnt = data_test.shape[1]
input_cnt = x_test_output.shape[1]

cols = [f'in{i}' if i < input_cnt else f'out{i}' for i in range(output_cnt)]

df = pd.DataFrame(data_test, columns=cols)
df.head()

df.to_csv('test.csv')

file_metadata = { 'name': 'test.csv', 'mimeType': 'text/csv' }
media = MediaFileUpload('test.csv', mimetype='text/csv', resumable=True)
created = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 1WwInc3YPCw8ZUHmZms6oNjSIFBLr2RUd


In [44]:
output_cnt = data_validation.shape[1]
input_cnt = x_validation_output.shape[1]

cols = [f'in{i}' if i < input_cnt else f'out{i}' for i in range(output_cnt)]

df = pd.DataFrame(data_validation, columns=cols)
df.head()

df.to_csv('validation.csv')

file_metadata = { 'name': 'validation.csv', 'mimeType': 'text/csv' }
media = MediaFileUpload('validation.csv', mimetype='text/csv', resumable=True)
created = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
print('File ID: {}'.format(created.get('id')))

File ID: 1e6RrqxJZvaOu7kwkItjb5OAa_E6grEpK


In [0]:
to_download = {    
    'train.csv' : '10ObsG1CBpM6TM79h-Z6qJ04Xv44-iwBE',
    'test.csv' : '1WwInc3YPCw8ZUHmZms6oNjSIFBLr2RUd',
    'validation.csv' : '1e6RrqxJZvaOu7kwkItjb5OAa_E6grEpK'
}

for filename, fileid in to_download.items():
  drive.CreateFile({'id': fileid}).GetContentFile(filename)